<a href="https://colab.research.google.com/github/MatteoOnger/algo-collusion-mm/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Collusion in Market Making - EXP3

A notebook testing various EXP3 agents implementing market-making strategies in the Glosten-Milgrom environment.

## Notebook Initialization

### Colab Environment Setup

In [ ]:
# Do NOT run this cell in local environment - it's intended for Google Colab only.

# Clone GitHub repository
!git clone https://github.com/MatteoOnger/algo-collusion-mm.git

# Install dependencies
!pip install --quiet -r /content/algo-collusion-mm/requirements.txt

# Set working directory
%cd /content/algo-collusion-mm

### Local Environment Setup

In [ ]:
# Do NOT run this cell in Google Colab - it's intended for local Jupyter Notebooks only.

# Autoreload imports
%load_ext autoreload
%autoreload 2

# Select interactive backend for matplotlib
%matplotlib widget

## Main Execution

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import time

import src.utils.plots as plots
import src.utils.storage as storage

from datetime import datetime

from src.agents.agent import Agent
from src.agents.makers.exp3 import MakerEXP3
from src.agents.traders.nopass import NoPassTrader
from src.envs import GMEnv
from src.utils.stats import OnlineVectorStats

In [ ]:
def scale_rewards_array(
    cumulative_rewards: np.ndarray|float,
    n_agents: int = 2,
    n_episodes: int = 1,
    min_reward: float = -0.5,
    max_reward: float =  0.5
) -> np.ndarray:
    """
    Scales a NumPy array of cumulative rewards from the range [r_min, r_max] to [-1, +1].

    Parameters
    ----------
    cumulative_rewards : np.ndarray or float
        Array of cumulative rewards to be scaled.
    n_agents : int, default=2
        Number of agents.
    n_episodes : int, default=1
        Number of episodes used to compute the cumulative rewards.
    min_reward : float, default=-0.5
        Expected minimum reward per episode in the single-agent case.
    max_reward : float, default=0.5
        Expected maximum reward per episode in the single-agent case.

    Returns
    -------
    : np.ndarray
        Array of rewards scaled to the range [-1, +1].

    Raises
    ------
    ValueError
        If min_reward and max_reward are equal (to avoid division by zero).
    """
    if min_reward == max_reward:
        raise ValueError('min_reward and max_reward must be different to avoid division by zero')

    r_min = n_episodes * min_reward
    r_max = n_episodes * max_reward / n_agents

    scaled = -1 + (cumulative_rewards - r_min) * 2 / (r_max - r_min)
    return scaled


def split_array(arr: np.ndarray, window_size: int) -> np.ndarray:
    """
    Split an array into sub-arrays of fixed window size along the last axis.

    If `window_size` is non-positive, the array is reshaped so that the last
    axis becomes a single window of length equal to its size.
    This is useful, for example, to ensure a consistent 3D shape
    when no actual splitting is performed.

    Parameters
    ----------
    arr : np.ndarray
        Input array to be split.
    window_size : int
        Size of each window. Must be a positive integer.
        If <= 0, the array is reshaped to (..., 1, N), where N is the
        original length of the last axis.

    Returns
    -------
    : np.ndarray
        Reshaped array with shape (..., n_windows, window_size).
        If `window_size <= 0`, returns the original array.

    Raises
    ------
    ValueError
        If `window_size` is not a divisor of the length of the last axis.
    """
    if window_size <= 0:
        return arr.reshape(arr.shape[:-1] + (1, -1))
    return arr.reshape(arr.shape[:-1] + (-1, window_size))


def get_calvano_collusion_index(rewards: np.ndarray, nash_reward: float, coll_reward: float, window_size: int = 0) -> np.ndarray:
    """
    Compute the Calvano Collusion Index (CCI) from agent rewards.

    The CCI measures the degree of collusion relative to Nash equilibrium
    and perfect collusion benchmarks. Rewards are optionally aggregated
    over fixed-size windows before computing the index.

    Parameters
    ----------
    rewards : np.ndarray
        Array of shape (n_agents, n_episodes) containing per-agent rewards.
    nash_reward : float
        Benchmark reward under Nash equilibrium (total across all agents).
    coll_reward : float
        Benchmark reward under perfect collusion (total across all agents).
    window_size : int, default=0
        Size of the episode window for reward aggregation.
        If 0, no windowing is applied.

    Returns
    -------
    : np.ndarray
        Array of CCI values per agent and per window.

    See Also
    --------
    - Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020).
    Artificial intelligence, algorithmic pricing, and collusion.
    *American Economic Review, 110*(10), 3267–3297.
    https://doi.org/10.1257/aer.20190623
    """
    nash_reward /= len(rewards)
    coll_reward /= len(rewards)

    rewards = split_array(rewards, window_size)
    avg_rewards = rewards.mean(axis=-1)

    cci = (avg_rewards - nash_reward) / (coll_reward - nash_reward)
    return cci

### Load Agents

In [ ]:
saver = storage.ExperimentStorage('./experiments')

objects = saver.load_objects('')
print(f'Objects loaded: {list(objects.keys())}')

### Multiple Runs

In [ ]:
epsilons = np.arange(1, 100) / 100

In [ ]:
r = 99              # Number of experiments
n = 50_000          # Number of episodes
k = 100             # Number of windows
w = n // k          # Window size

nash_reward = 0.1   # Nash reward
coll_reward = 0.5   # Collusive reward

# Prices and action space of the market makers
action_space = np.array([[0., 0.], [.6, .4], [1., 0.]])

# Min CCI of the last window per each experiment
final_cci = np.zeros(r)
# Min cumulative rewards of the last window per each experiment
final_cum_rewards = np.zeros(r)

# To save experimental results
saver = storage.ExperimentStorage('./experiments/exp3/varying_epsilon_2/50k')
# To compute online statistics
stats_cci = OnlineVectorStats(2)

start_time = time.time()
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
saver.print_and_save(f'Started at {current_time}')

for i in range(r):
    if i % 10 == 0:
        saver.print_and_save(f'Running {i} ...')

    agents: dict[str, Agent] = {
        'maker_u_0': MakerEXP3(epsilon=epsilons[i], action_space=action_space, name='maker_u_0'),
        'maker_u_1': MakerEXP3(epsilon=epsilons[i], action_space=action_space, name='maker_u_1'),
        'trader_0': NoPassTrader(name='trader_0'),
    }

    env = GMEnv(
        generate_vt = lambda: 0.5,
        n_episodes = n,
        n_makers_u = 2,
        n_makers_i = 0,
        n_traders = 1,
    )

    _, info = env.reset()

    for agent in env.agent_iter():
        action = agents[agent].act(env.observe(agent))
        _, rewards, _, _, infos = env.step(action)

        if infos['episode_finished']:
            for a in env.possible_agents:
                agents[a].update(rewards[a], infos[a])

    cci = get_calvano_collusion_index(
        np.array([agent.history.get_rewards() for name, agent in agents.items() if name in env.makers]),
        nash_reward = nash_reward,
        coll_reward = coll_reward,
        window_size = w
    )

    info = {
        'parmas' : {
            'n_episodes' : n,
            'window_size' : w,
            'action_space' : str(action_space).replace('\n', ','),
            'epsilon' : epsilons[i],
            'agent_type' : [agent.__class__.__name__ for agent in agents.values()]
        },
        'most_common_action' : {
            n//w : {name : str(agent.history.compute_most_common(slice(-w, None))) for name, agent in agents.items() if name in env.makers}
        },
        'cumulative_rewards' : {
            0  : {name : round(float(agent.history.get_rewards(slice(0, w)).sum()), 3) for name, agent in agents.items()},
            n//w : {name : round(float(agent.history.get_rewards(slice(-w, None)).sum()), 3) for name, agent in agents.items()},
            'global' : env.cumulative_rewards
        },
        'cci' : {
            0  : {name : round(float(cci[idx, 0]), 3) for idx, name in enumerate(env.makers)},
            n//w  : {name : round(float(cci[idx, -1]), 3) for idx, name in enumerate(env.makers)},
            'global' : {name : round(float(cci[idx, :].mean()), 3) for idx, name in enumerate(env.makers)},
        },
        'seed' : {
            name : agent._seed for name, agent in agents.items()
        }
    }

    stats_cci.update(cci[:, -1])
    final_cci[i] = cci[:, -1].min()
    final_cum_rewards[i] = np.array([agent.history.get_rewards(slice(-w, None)).sum() for name, agent in agents.items() if name in env.makers]).min()
    
    dir = saver.save_experiment([env] + list(agents.values()), info=info)
    saver.print_and_save(f'{i:03} {"*" if cci[0, -1] >= 0.9 or cci[1, -1] >= 0.9 else " "} -> CCI:{info["cci"][n//w]}'.ljust(60) + f' ({dir})')

end_time = time.time()
execution_time = end_time - start_time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
saver.print_and_save(f'Done at {current_time} | Execution time: {execution_time:.2f} seconds')

saver.save_objects({'final_cci': final_cci, 'final_cum_rewards': final_cum_rewards})
saver.print_and_save(
    f'Results:\n'
    f'- [CCI] Average in the last window: {np.round(stats_cci.get_mean(), 4)}\n'
    f'- [CCI] Standard deviation in the last window: {np.round(stats_cci.get_std(), 4)}'
)

### Single Run

In [ ]:
saver = storage.ExperimentStorage('./experiments')

In [ ]:
n = 10_000       # Number of episodes
w = 50           # Window size
k = n // w       # Number of windows

nash_reward = 0.1  # Nash reward
coll_reward = 0.5  # Collusive reward

counter = 0        # Number of episodes done

# Prices and action space of the market makers
action_space = np.array([[0.0, 0.0], [.6, .4], [1., 0.]])

agents: dict[str, Agent] = {
    'maker_u_0': MakerEXP3(epsilon=MakerEXP3.compute_epsilon(len(action_space), n), action_space=action_space, name='maker_u_0'),
    'maker_u_1': MakerEXP3(epsilon=MakerEXP3.compute_epsilon(len(action_space), n), action_space=action_space, name='maker_u_1'),
    'trader_0': NoPassTrader(name='trader_0'),
}

env = GMEnv(
    generate_vt = lambda: 0.5,
    n_episodes = n,
    n_makers_u = 2,
    n_makers_i = 0,
    n_traders = 1,
)

start_time = time.time()
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Started at {current_time}')

_, info = env.reset()
for agent in env.agent_iter():
    action = agents[agent].act(env.observe(agent))
    _, rewards, _, _, infos = env.step(action)

    if infos['episode_finished']:
        if counter % 10_000 == 0:
            print(f'Running episode {counter} ...')

        for a in env.possible_agents:
            if a in env.makers and counter % (k//5) == 0:
                agents[a].history.record_extra(agents[a].weights.copy())
            
            agents[a].update(rewards[a], infos[a])
    
        counter += 1

end_time = time.time()
execution_time = end_time - start_time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Done at {current_time} | Execution time: {execution_time:.2f} seconds')

cci = get_calvano_collusion_index(
    np.array([agent.history.get_rewards() for name, agent in agents.items() if name in env.makers]),
    nash_reward = nash_reward,
    coll_reward = coll_reward,
    window_size = w
)

info = {
    'parmas' : {
        'n_episodes' : n,
        'window_size' : w,
        'action_space' : str(action_space).replace('\n', ','),
        'agent_type' : [agent.__class__.__name__ for agent in agents.values()],
    },
    'most_common_action' : {
        n//w : {name : str(agent.history.compute_most_common(slice(-w, None))) for name, agent in agents.items() if name in env.makers}
    },
    'cumulative_rewards' : {
        0  : {name : round(float(agent.history.get_rewards(slice(0, w)).sum()), 3) for name, agent in agents.items()},
        n//w : {name : round(float(agent.history.get_rewards(slice(-w, None)).sum()), 3) for name, agent in agents.items()},
        'global' : env.cumulative_rewards
    },
    'cci' : {
        0  : {name : round(float(cci[idx, 0]), 3) for idx, name in enumerate(env.makers)},
        n//w  : {name : round(float(cci[idx, -1]), 3) for idx, name in enumerate(env.makers)},
        'global' : {name : round(float(cci[idx, :].mean()), 3) for idx, name in enumerate(env.makers)},
    },
    'seed' : {
        name : agent._seed for name, agent in agents.items()
    }
}

fig = plots.plot_all(
    window_size = w,
    makers = {name:agent for name, agent in agents.items() if name in env.makers},
    cci = cci,
    makers_belif = {name:agent.weights for name, agent in agents.items() if name in env.makers},
    nash_reward = nash_reward,
    coll_reward = coll_reward
)

dir =  saver.save_experiment([env] + list(agents.values()), fig, info)

print(json.dumps(info, indent=2))
display(fig)
print(dir)

### Additional Plots

In [ ]:
saver = storage.ExperimentStorage('./experiments/exp3/varying_epsilon')

objs = saver.load_objects('./experiments/exp3/varying_epsilon/1k')
final_cci_1k, final_cum_rewards_1k = objs['final_cci'], scale_rewards_array(objs['final_cum_rewards'],  n_episodes=10)

objs = saver.load_objects('./experiments/exp3/varying_epsilon/10k')
final_cci_10k, final_cum_rewards_10k = objs['final_cci'], scale_rewards_array(objs['final_cum_rewards'], n_episodes=100)

objs = saver.load_objects('./experiments/exp3/varying_epsilon/20k')
final_cci_20k, final_cum_rewards_20k = objs['final_cci'], scale_rewards_array(objs['final_cum_rewards'], n_episodes=200)

objs = saver.load_objects('./experiments/exp3/varying_epsilon/50k')
final_cci_50k, final_cum_rewards_50k = objs['final_cci'], scale_rewards_array(objs['final_cum_rewards'], n_episodes=500)

figure, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(epsilons, final_cci_1k, color='#1f77b4', label='1k episodes')
ax1.plot(epsilons, final_cci_10k, color='#ff7f0e', label='10k episodes')
ax1.plot(epsilons, final_cci_20k, color='#18d0f5', label='20k episodes')
ax1.plot(epsilons, final_cci_50k, color='#a767e3', label='50k episodes')

ax2.scatter(epsilons, final_cum_rewards_1k, color='red', alpha=0.5, s=2, marker='o', edgecolor='none')
ax2.scatter(epsilons, final_cum_rewards_10k, color='red', alpha=0.5, s=2, marker='o', edgecolor='none')
ax2.scatter(epsilons, final_cum_rewards_20k, color='red', alpha=0.5, s=2, marker='o', edgecolor='none')
ax2.scatter(epsilons, final_cum_rewards_50k, color='red', alpha=0.5, s=2, marker='o', edgecolor='none')

ax1.axhline(0, linestyle='--', color='black', label='Nash')
ax1.axhline(1, linestyle='--', color='green', label='Coll')

ax1.axvline(MakerEXP3.compute_epsilon(3,  1_000*0.25), linestyle=':', color='#1f77b4')
ax1.axvline(MakerEXP3.compute_epsilon(3, 10_000*0.25), linestyle=':', color='#ff7f0e')
ax1.axvline(MakerEXP3.compute_epsilon(3, 20_000*0.25), linestyle=':', color='#18d0f5')
ax1.axvline(MakerEXP3.compute_epsilon(3, 50_000*0.25), linestyle=':', color='#a767e3')

ax1.set_xlabel('Epsilons')
ax1.set_ylabel('CCI')
ax2.set_ylabel('Scaled Cumulative Reward')
ax1.set_title('(Min) CCI and Cumulative Rewards wrt Epsilon')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='best', ncol=3)
ax1.grid(True)

plt.show()

saver.save_figures({'figure': figure})